In [3]:
''' Read book text preprocessed by Linux commands '''

import numpy

with open ('Supplement-A.txt', 'r', encoding = 'UTF-8') as inFile:
    RawPragraphList = inFile.readlines()
RawParagraphList = [s.replace('\n', '').strip() for s in RawPragraphList]

In [4]:
import nltk.data

#nltk.download('punkt')
senTokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
SentenceList = [] # corpus
for para in RawParagraphList:
    SentenceList += senTokenizer.tokenize(para)

In [6]:
SentenceList

['SUPPLEMENT',
 'A DECISION MAKING MODELS Operations managers',
 'make many decisions as they manage processes and supply chains.',
 'Although the specifics of each situation vary, decision making generally involves the same basic steps: (1) recognize and clearly define the problem, (2) collect the information needed to analyze possible alternatives, and (3) choose and implement the most feasible alternative.',
 'Sometimes, hard thinking in a quiet room is sufficient.',
 'At other times, interacting with others or using more formal procedures are needed.',
 'Here, we present four such formal procedures: break-even analysis, the preference matrix, decision theory, and the decision tree.',
 'Break-even analysis helps the manager identify how much change in volume or demand is necessary before a second alternative becomes better than the first alternative.',
 'The preference matrix helps a manager deal with multiple criteria that cannot be evaluated with a single measure of merit, such as

In [25]:
''' TSF '''

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy # average and standard deviation
from numpy import linalg # l2 norm
import math # check inf

def CosineSim (x, y, List_x, List_y):
    dotProduct = numpy.dot(List_x, List_y)
    norm_x = linalg.norm(List_x)
    norm_y = linalg.norm(List_y)
    if norm_x * norm_y > 0:
        return (dotProduct / norm_x * norm_y)
    else: return 0
    
K = 10 # the number of sentences in a block

countVectorizer = CountVectorizer()
TFList = countVectorizer.fit_transform(SentenceList).toarray()
tfidfTransformer = TfidfTransformer()
TFIDFList = tfidfTransformer.fit_transform(TFList).toarray()
TermList = countVectorizer.get_feature_names()

InnerSimList = [] # [i, innerSim] for each element; i starts from 0
OuterSimList = [] # [i, outerSim] for each element
DissimList = [] # [i, dissimilarity] for each element

for i in range(K - 1, (len(TFList) - K + 1)):
    innerSim = 0
    outerSim = 0
    for a in range(i - K + 1, i + 1):
        for b in range(a + 1, i + 1):
            if a != b:
                innerSim += CosineSim(a, b, TFList[a], TFList[b])
        for c in range(i + 1, i + K):
            outerSim += CosineSim(a, c, TFList[a], TFList[c])
    if innerSim == 0:
        continue
    dissim = (innerSim - outerSim) / innerSim
    if math.isinf(dissim):
        print ('inner:', innerSim)
        print ('outer:', outerSim)
        continue
    InnerSimList.append([i, innerSim])
    OuterSimList.append([i, outerSim])
    DissimList.append({'i': i, 'dissim': dissim})

avg_dissim = numpy.mean([d['dissim'] for d in DissimList])
dev_dissim = numpy.std([d['dissim'] for d in DissimList])
print ('Avg =', avg_dissim)
print ('Dev =', dev_dissim)

Avg = -2.46714120253
Dev = 8.18311466512


In [26]:
DISSIM_THRESHOLD = avg_dissim + 0.25 * dev_dissim # the threshold of the dissimilarity

current_i = 0
ParagrahList = []
for d in DissimList:
    if d['dissim'] > DISSIM_THRESHOLD:
        ParagrahList.append(' '.join(SentenceList[current_i:d['i'] + 1]))
        current_i = d['i'] + 1
ParagrahList.append(' '.join(SentenceList[current_i:]))

In [27]:
print (len(ParagrahList))
for p in ParagrahList:
    print (p + '\n\n')

447
SUPPLEMENT A DECISION MAKING MODELS Operations managers make many decisions as they manage processes and supply chains. Although the specifics of each situation vary, decision making generally involves the same basic steps: (1) recognize and clearly define the problem, (2) collect the information needed to analyze possible alternatives, and (3) choose and implement the most feasible alternative. Sometimes, hard thinking in a quiet room is sufficient. At other times, interacting with others or using more formal procedures are needed. Here, we present four such formal procedures: break-even analysis, the preference matrix, decision theory, and the decision tree. Break-even analysis helps the manager identify how much change in volume or demand is necessary before a second alternative becomes better than the first alternative. The preference matrix helps a manager deal with multiple criteria that cannot be evaluated with a single measure of merit, such as total profit or cost. Decisio